This notebook serves to experiment with different ways of ensembling the fine-tuned models for better prediction performance. 

In [1]:
# Instalación de paquetes necesarios
!pip3 install sentencepiece
!pip3 install pytorch-lightning
!pip3 install --upgrade accelerate
!pip3 install emoji
!pip3 install framework-reproducibility
!pip3 install transformers
!pip3 install sacremoses # for data augmentation by back-translation
!pip3 install -U easynmt # for data augmentation by back-translation
!pip3 install transformers datasets
!pip3 install contractions
!pip3 install tensorflow
!pip3 install scikit-learn
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install sklearn-hierarchical-classification # para ejecutar subtask_1_2a.py
!pip3 install tf-keras
!pip3 install optuna

In [2]:
# Importar librerías
import json
import numpy as np
import os
import subprocess

# order of label that was used for model fine-tuning, has to be the same order for zipping with the predicted probabilities
labels = ['Appeal to authority', 'Appeal to fear/prejudice', 'Bandwagon',
          'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification',
          'Doubt', 'Exaggeration/Minimisation', 'Flag-waving',
          'Glittering generalities (Virtue)', 'Loaded Language',
          "Misrepresentation of Someone's Position (Straw Man)",
          'Name calling/Labeling',
          'Obfuscation, Intentional vagueness, Confusion',
          'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum',
          'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché',
          'Whataboutism']

# set up root path
root_path = os.getcwd()

# path to the datasets
data_path = f'{root_path}/data/'

# path to the models
model_path = f'{root_path}/fine_tuned_models/'

# Use the labels of the predicted development set for ensembling

# On the models fine-tuned using the original training dataset

In [3]:
model_checkpoints = [
        'google-bert/bert-base-uncased',
        'microsoft/deberta-v3-base',
        'FacebookAI/roberta-base',
        'distilbert/distilbert-base-uncased',
        'xlnet/xlnet-base-cased',
        'openai-community/gpt2',
        ]

In [4]:
model_prediction_dict = {}
train_df_name = 'original_train'

for model_name in model_checkpoints:
    model_name_path = f"{model_path}{model_name}/"
    if os.path.exists(f'{model_name_path}{train_df_name}/config.json'):
        with open(f'{model_name_path}{train_df_name}/dev_predicted_labels.json','r') as f:
            model_prediction_dict[model_name] = json.load(f)
model_prediction_dict

{'google-bert/bert-base-uncased': [{'id': '63292',
   'labels': ['Black-and-white Fallacy/Dictatorship']},
  {'id': '70419',
   'labels': ['Black-and-white Fallacy/Dictatorship',
    'Loaded Language',
    'Name calling/Labeling']},
  {'id': '63673', 'labels': ['Smears']},
  {'id': '71297', 'labels': []},
  {'id': '66340', 'labels': ['Smears']},
  {'id': '77304', 'labels': []},
  {'id': '76719', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '78621', 'labels': ['Appeal to authority']},
  {'id': '64354', 'labels': ['Loaded Language']},
  {'id': '69527',
   'labels': ['Exaggeration/Minimisation',
    'Flag-waving',
    'Glittering generalities (Virtue)',
    'Name calling/Labeling']},
  {'id': '63834', 'labels': ['Smears']},
  {'id': '63225', 'labels': ['Appeal to authority', 'Smears']},
  {'id': '78675', 'labels': []},
  {'id': '66933', 'labels': ['Loaded Language']},
  {'id': '78661', 'labels': ['Flag-waving', 'Name calling/Labeling']},
  {'id': '69180',
   'labels': ['Appeal

## Ensemble by union

This method of ensembling simply takes the union of the labels predicted by all models and use that union as the predicted labels.

In [5]:
ensemble_by_union = {}
# an abitrary list to iterate through the predictions
prediction_list = list(model_prediction_dict.values())[0]

for i, ele in enumerate(prediction_list):
    id = ele.get('id')
    ens_pred_labels = set()
    for model_name, predictions in model_prediction_dict.items():
        pred = predictions[i]
        pred_id = pred.get('id')
        pred_labels = pred.get('labels')
        ens_pred_labels = ens_pred_labels.union(set(pred_labels))
    if id != pred_id:
        print('No, wrong index do not append!')
    ensemble_by_union[pred_id] = list(ens_pred_labels)

In [6]:
prediction_list

[{'id': '63292', 'labels': ['Black-and-white Fallacy/Dictatorship']},
 {'id': '70419',
  'labels': ['Black-and-white Fallacy/Dictatorship',
   'Loaded Language',
   'Name calling/Labeling']},
 {'id': '63673', 'labels': ['Smears']},
 {'id': '71297', 'labels': []},
 {'id': '66340', 'labels': ['Smears']},
 {'id': '77304', 'labels': []},
 {'id': '76719', 'labels': ['Name calling/Labeling', 'Smears']},
 {'id': '78621', 'labels': ['Appeal to authority']},
 {'id': '64354', 'labels': ['Loaded Language']},
 {'id': '69527',
  'labels': ['Exaggeration/Minimisation',
   'Flag-waving',
   'Glittering generalities (Virtue)',
   'Name calling/Labeling']},
 {'id': '63834', 'labels': ['Smears']},
 {'id': '63225', 'labels': ['Appeal to authority', 'Smears']},
 {'id': '78675', 'labels': []},
 {'id': '66933', 'labels': ['Loaded Language']},
 {'id': '78661', 'labels': ['Flag-waving', 'Name calling/Labeling']},
 {'id': '69180',
  'labels': ['Appeal to fear/prejudice',
   'Black-and-white Fallacy/Dictatorshi

In [7]:
ensemble_by_union

{'63292': ['Repetition', 'Black-and-white Fallacy/Dictatorship', 'Doubt'],
 '70419': ['Loaded Language',
  "Misrepresentation of Someone's Position (Straw Man)",
  'Smears',
  'Black-and-white Fallacy/Dictatorship',
  'Name calling/Labeling'],
 '63673': ['Smears'],
 '71297': [],
 '66340': ['Loaded Language', 'Smears'],
 '77304': [],
 '76719': ['Appeal to authority', 'Name calling/Labeling', 'Smears'],
 '78621': ['Black-and-white Fallacy/Dictatorship',
  'Slogans',
  'Appeal to authority',
  'Smears'],
 '64354': ['Black-and-white Fallacy/Dictatorship',
  'Loaded Language',
  'Slogans'],
 '69527': ['Loaded Language',
  'Smears',
  'Glittering generalities (Virtue)',
  'Flag-waving',
  'Name calling/Labeling',
  'Exaggeration/Minimisation'],
 '63834': ['Smears'],
 '63225': ['Smears', 'Appeal to authority'],
 '78675': ['Smears'],
 '66933': ['Loaded Language'],
 '78661': ['Black-and-white Fallacy/Dictatorship',
  'Flag-waving',
  'Name calling/Labeling',
  'Exaggeration/Minimisation'],
 '69

In [8]:
with open(f'{model_path}ensemble_by_union/{train_df_name}/dev_predicted_labels.json', 'w') as f:
        json.dump([{'id': str(k), 'labels': v} for k, v in ensemble_by_union.items()], f, indent=4)

In [9]:
# perform the SemEval hierarchical evaluation on the predicted labels and save the output in eval folder
file_ = open(f'{model_path}ensemble_by_union/{train_df_name}/SemEval_hierarchical_eval.txt', "w")
subprocess.Popen(f'python3 ./data/scorer-baseline/subtask_1_2a.py -g ./data/dev_gold_labels/dev_subtask1_en.json -p ./fine_tuned_models/ensemble_by_union/{train_df_name}/dev_predicted_labels.json',shell=True,stdout=file_)
file_.close()

## Ensemble by vote

This method of ensembling counts how many times a given label has been predicted by different models. If the label receives a majority vote (>=0.5), it will be included as the predicted labels.

In [10]:
model_prediction_dict

{'google-bert/bert-base-uncased': [{'id': '63292',
   'labels': ['Black-and-white Fallacy/Dictatorship']},
  {'id': '70419',
   'labels': ['Black-and-white Fallacy/Dictatorship',
    'Loaded Language',
    'Name calling/Labeling']},
  {'id': '63673', 'labels': ['Smears']},
  {'id': '71297', 'labels': []},
  {'id': '66340', 'labels': ['Smears']},
  {'id': '77304', 'labels': []},
  {'id': '76719', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '78621', 'labels': ['Appeal to authority']},
  {'id': '64354', 'labels': ['Loaded Language']},
  {'id': '69527',
   'labels': ['Exaggeration/Minimisation',
    'Flag-waving',
    'Glittering generalities (Virtue)',
    'Name calling/Labeling']},
  {'id': '63834', 'labels': ['Smears']},
  {'id': '63225', 'labels': ['Appeal to authority', 'Smears']},
  {'id': '78675', 'labels': []},
  {'id': '66933', 'labels': ['Loaded Language']},
  {'id': '78661', 'labels': ['Flag-waving', 'Name calling/Labeling']},
  {'id': '69180',
   'labels': ['Appeal

In [11]:
ensemble_by_vote = {}
total_model = len(model_prediction_dict.keys())
# an abitrary list to iterate through the predictions
prediction_list = list(model_prediction_dict.values())[0]

for i, ele in enumerate(prediction_list):
    id = ele.get('id')
    ens_pred_labels = []
    pred_label_count = {}
    for model_name, predictions in model_prediction_dict.items():
        pred = predictions[i]
        pred_id = pred.get('id')
        pred_labels = pred.get('labels')
        for label in pred_labels:
            pred_label_count[label] = pred_label_count.get(label,0) + 1
    if id != pred_id:
        print('No, wrong index do not append!')
    print(pred_label_count)
    ensemble_by_vote[pred_id] = [label for label, count in pred_label_count.items() if count / total_model >= 0.5]

{'Black-and-white Fallacy/Dictatorship': 6, 'Repetition': 1, 'Doubt': 1}
{'Black-and-white Fallacy/Dictatorship': 3, 'Loaded Language': 5, 'Name calling/Labeling': 6, "Misrepresentation of Someone's Position (Straw Man)": 1, 'Smears': 1}
{'Smears': 5}
{}
{'Smears': 6, 'Loaded Language': 2}
{}
{'Name calling/Labeling': 6, 'Smears': 6, 'Appeal to authority': 1}
{'Appeal to authority': 3, 'Smears': 3, 'Black-and-white Fallacy/Dictatorship': 1, 'Slogans': 1}
{'Loaded Language': 5, 'Black-and-white Fallacy/Dictatorship': 2, 'Slogans': 1}
{'Exaggeration/Minimisation': 2, 'Flag-waving': 1, 'Glittering generalities (Virtue)': 1, 'Name calling/Labeling': 6, 'Loaded Language': 2, 'Smears': 3}
{'Smears': 6}
{'Appeal to authority': 6, 'Smears': 6}
{'Smears': 1}
{'Loaded Language': 6}
{'Flag-waving': 5, 'Name calling/Labeling': 5, 'Black-and-white Fallacy/Dictatorship': 2, 'Exaggeration/Minimisation': 1}
{'Appeal to fear/prejudice': 1, 'Black-and-white Fallacy/Dictatorship': 5, 'Flag-waving': 2, 'L

In [12]:
ensemble_by_vote

{'63292': ['Black-and-white Fallacy/Dictatorship'],
 '70419': ['Black-and-white Fallacy/Dictatorship',
  'Loaded Language',
  'Name calling/Labeling'],
 '63673': ['Smears'],
 '71297': [],
 '66340': ['Smears'],
 '77304': [],
 '76719': ['Name calling/Labeling', 'Smears'],
 '78621': ['Appeal to authority', 'Smears'],
 '64354': ['Loaded Language'],
 '69527': ['Name calling/Labeling', 'Smears'],
 '63834': ['Smears'],
 '63225': ['Appeal to authority', 'Smears'],
 '78675': [],
 '66933': ['Loaded Language'],
 '78661': ['Flag-waving', 'Name calling/Labeling'],
 '69180': ['Black-and-white Fallacy/Dictatorship',
  'Loaded Language',
  'Name calling/Labeling'],
 '78636': [],
 '68040': ['Black-and-white Fallacy/Dictatorship',
  'Thought-terminating cliché',
  'Loaded Language'],
 '75948': ['Appeal to authority', 'Loaded Language', 'Smears'],
 '76218': [],
 '66685': ['Slogans'],
 '69589': [],
 '64559': ['Doubt',
  'Repetition',
  'Thought-terminating cliché',
  'Whataboutism'],
 '66414': [],
 '63891

In [13]:
with open(f'{model_path}ensemble_by_vote/{train_df_name}/dev_predicted_labels.json', 'w') as f:
        json.dump([{'id': str(k), 'labels': v} for k, v in ensemble_by_vote.items()], f, indent=4)

In [14]:
# perform the SemEval hierarchical evaluation on the predicted labels and save the output in eval folder
file_ = open(f'{model_path}ensemble_by_vote/{train_df_name}/SemEval_hierarchical_eval.txt', "w")
subprocess.Popen(f'python3 ./data/scorer-baseline/subtask_1_2a.py -g ./data/dev_gold_labels/dev_subtask1_en.json -p ./fine_tuned_models/ensemble_by_vote/{train_df_name}/dev_predicted_labels.json',shell=True,stdout=file_)
file_.close()

# On the models fine-tuned using the balanced training dataset

In [15]:
model_prediction_dict = {}
train_df_name = 'balanced_train'

for model_name in model_checkpoints:
    model_name_path = f"{model_path}{model_name}/"
    if os.path.exists(f'{model_name_path}{train_df_name}/config.json'):
        with open(f'{model_name_path}{train_df_name}/dev_predicted_labels.json','r') as f:
            model_prediction_dict[model_name] = json.load(f)
model_prediction_dict

{'google-bert/bert-base-uncased': [{'id': '63292',
   'labels': ['Black-and-white Fallacy/Dictatorship',
    'Thought-terminating cliché']},
  {'id': '70419', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '63673', 'labels': []},
  {'id': '71297', 'labels': []},
  {'id': '66340', 'labels': []},
  {'id': '77304', 'labels': []},
  {'id': '76719', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '78621',
   'labels': ['Appeal to authority', 'Black-and-white Fallacy/Dictatorship']},
  {'id': '64354', 'labels': []},
  {'id': '69527', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '63834', 'labels': []},
  {'id': '63225', 'labels': ['Appeal to authority', 'Smears']},
  {'id': '78675', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '66933', 'labels': ['Loaded Language']},
  {'id': '78661', 'labels': ['Flag-waving']},
  {'id': '69180',
   'labels': ['Black-and-white Fallacy/Dictatorship', 'Loaded Language']},
  {'id': '78636', 'labels': ['Doubt']},
  {'id'

## Ensemble by union

This method of ensembling simply takes the union of the labels predicted by all models and use that union as the predicted labels.

In [16]:
ensemble_by_union = {}
# an abitrary list to iterate through the predictions
prediction_list = list(model_prediction_dict.values())[0]

for i, ele in enumerate(prediction_list):
    id = ele.get('id')
    ens_pred_labels = set()
    for model_name, predictions in model_prediction_dict.items():
        pred = predictions[i]
        pred_id = pred.get('id')
        pred_labels = pred.get('labels')
        ens_pred_labels = ens_pred_labels.union(set(pred_labels))
    if id != pred_id:
        print('No, wrong index do not append!')
    ensemble_by_union[pred_id] = list(ens_pred_labels)

In [17]:
prediction_list

[{'id': '63292',
  'labels': ['Black-and-white Fallacy/Dictatorship',
   'Thought-terminating cliché']},
 {'id': '70419', 'labels': ['Name calling/Labeling', 'Smears']},
 {'id': '63673', 'labels': []},
 {'id': '71297', 'labels': []},
 {'id': '66340', 'labels': []},
 {'id': '77304', 'labels': []},
 {'id': '76719', 'labels': ['Name calling/Labeling', 'Smears']},
 {'id': '78621',
  'labels': ['Appeal to authority', 'Black-and-white Fallacy/Dictatorship']},
 {'id': '64354', 'labels': []},
 {'id': '69527', 'labels': ['Name calling/Labeling', 'Smears']},
 {'id': '63834', 'labels': []},
 {'id': '63225', 'labels': ['Appeal to authority', 'Smears']},
 {'id': '78675', 'labels': ['Name calling/Labeling', 'Smears']},
 {'id': '66933', 'labels': ['Loaded Language']},
 {'id': '78661', 'labels': ['Flag-waving']},
 {'id': '69180',
  'labels': ['Black-and-white Fallacy/Dictatorship', 'Loaded Language']},
 {'id': '78636', 'labels': ['Doubt']},
 {'id': '68040',
  'labels': ['Black-and-white Fallacy/Dictat

In [18]:
ensemble_by_union

{'63292': ['Black-and-white Fallacy/Dictatorship',
  'Thought-terminating cliché'],
 '70419': ['Loaded Language',
  'Thought-terminating cliché',
  'Name calling/Labeling',
  'Smears'],
 '63673': ['Loaded Language'],
 '71297': [],
 '66340': ['Glittering generalities (Virtue)'],
 '77304': ['Flag-waving'],
 '76719': ['Loaded Language',
  'Appeal to authority',
  'Name calling/Labeling',
  'Smears'],
 '78621': ['Black-and-white Fallacy/Dictatorship',
  'Smears',
  'Appeal to authority'],
 '64354': ['Repetition',
  'Black-and-white Fallacy/Dictatorship',
  'Loaded Language'],
 '69527': ['Loaded Language', 'Name calling/Labeling', 'Smears'],
 '63834': ['Glittering generalities (Virtue)', 'Smears'],
 '63225': ['Loaded Language', 'Appeal to authority', 'Smears'],
 '78675': ['Name calling/Labeling', 'Exaggeration/Minimisation', 'Smears'],
 '66933': ['Black-and-white Fallacy/Dictatorship',
  'Loaded Language',
  'Thought-terminating cliché'],
 '78661': ['Black-and-white Fallacy/Dictatorship',
 

In [19]:
with open(f'{model_path}ensemble_by_union/{train_df_name}/dev_predicted_labels.json', 'w') as f:
        json.dump([{'id': str(k), 'labels': v} for k, v in ensemble_by_union.items()], f, indent=4)

In [20]:
# perform the SemEval hierarchical evaluation on the predicted labels and save the output in eval folder
file_ = open(f'{model_path}ensemble_by_union/{train_df_name}/SemEval_hierarchical_eval.txt', "w")
subprocess.Popen(f'python3 ./data/scorer-baseline/subtask_1_2a.py -g ./data/dev_gold_labels/dev_subtask1_en.json -p ./fine_tuned_models/ensemble_by_union/{train_df_name}/dev_predicted_labels.json',shell=True,stdout=file_)
file_.close()

## Ensemble by vote

This method of ensembling counts how many times a given label has been predicted by different models. If the label receives a majority vote (>=0.5), it will be included as the predicted labels.

In [21]:
model_prediction_dict


{'google-bert/bert-base-uncased': [{'id': '63292',
   'labels': ['Black-and-white Fallacy/Dictatorship',
    'Thought-terminating cliché']},
  {'id': '70419', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '63673', 'labels': []},
  {'id': '71297', 'labels': []},
  {'id': '66340', 'labels': []},
  {'id': '77304', 'labels': []},
  {'id': '76719', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '78621',
   'labels': ['Appeal to authority', 'Black-and-white Fallacy/Dictatorship']},
  {'id': '64354', 'labels': []},
  {'id': '69527', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '63834', 'labels': []},
  {'id': '63225', 'labels': ['Appeal to authority', 'Smears']},
  {'id': '78675', 'labels': ['Name calling/Labeling', 'Smears']},
  {'id': '66933', 'labels': ['Loaded Language']},
  {'id': '78661', 'labels': ['Flag-waving']},
  {'id': '69180',
   'labels': ['Black-and-white Fallacy/Dictatorship', 'Loaded Language']},
  {'id': '78636', 'labels': ['Doubt']},
  {'id'

In [22]:
ensemble_by_vote = {}
total_model = len(model_prediction_dict.keys())
# an abitrary list to iterate through the predictions
prediction_list = list(model_prediction_dict.values())[0]

for i, ele in enumerate(prediction_list):
    id = ele.get('id')
    ens_pred_labels = []
    pred_label_count = {}
    for model_name, predictions in model_prediction_dict.items():
        pred = predictions[i]
        pred_id = pred.get('id')
        pred_labels = pred.get('labels')
        for label in pred_labels:
            pred_label_count[label] = pred_label_count.get(label,0) + 1
    if id != pred_id:
        print('No, wrong index do not append!')
    print(pred_label_count)
    ensemble_by_vote[pred_id] = [label for label, count in pred_label_count.items() if count / total_model >= 0.5]

{'Black-and-white Fallacy/Dictatorship': 3, 'Thought-terminating cliché': 1}
{'Name calling/Labeling': 5, 'Smears': 4, 'Loaded Language': 2, 'Thought-terminating cliché': 1}
{'Loaded Language': 1}
{}
{'Glittering generalities (Virtue)': 1}
{'Flag-waving': 2}
{'Name calling/Labeling': 6, 'Smears': 5, 'Loaded Language': 1, 'Appeal to authority': 1}
{'Appeal to authority': 3, 'Black-and-white Fallacy/Dictatorship': 1, 'Smears': 1}
{'Black-and-white Fallacy/Dictatorship': 3, 'Loaded Language': 3, 'Repetition': 1}
{'Name calling/Labeling': 6, 'Smears': 6, 'Loaded Language': 4}
{'Smears': 4, 'Glittering generalities (Virtue)': 2}
{'Appeal to authority': 6, 'Smears': 3, 'Loaded Language': 1}
{'Name calling/Labeling': 1, 'Smears': 1, 'Exaggeration/Minimisation': 2}
{'Loaded Language': 3, 'Thought-terminating cliché': 1, 'Black-and-white Fallacy/Dictatorship': 1}
{'Flag-waving': 3, 'Black-and-white Fallacy/Dictatorship': 3, 'Smears': 1, 'Thought-terminating cliché': 1, 'Name calling/Labeling': 

In [23]:
ensemble_by_vote

{'63292': ['Black-and-white Fallacy/Dictatorship'],
 '70419': ['Name calling/Labeling', 'Smears'],
 '63673': [],
 '71297': [],
 '66340': [],
 '77304': [],
 '76719': ['Name calling/Labeling', 'Smears'],
 '78621': ['Appeal to authority'],
 '64354': ['Black-and-white Fallacy/Dictatorship', 'Loaded Language'],
 '69527': ['Name calling/Labeling', 'Smears', 'Loaded Language'],
 '63834': ['Smears'],
 '63225': ['Appeal to authority', 'Smears'],
 '78675': [],
 '66933': ['Loaded Language'],
 '78661': ['Flag-waving', 'Black-and-white Fallacy/Dictatorship'],
 '69180': ['Black-and-white Fallacy/Dictatorship',
  'Loaded Language',
  'Name calling/Labeling',
  'Smears'],
 '78636': [],
 '68040': ['Black-and-white Fallacy/Dictatorship'],
 '75948': ['Appeal to authority', 'Loaded Language', 'Name calling/Labeling'],
 '76218': [],
 '66685': [],
 '69589': [],
 '64559': ['Repetition'],
 '66414': [],
 '63891': ['Name calling/Labeling', 'Smears', 'Loaded Language'],
 '70213': [],
 '77611': ['Name calling/Lab

In [24]:
with open(f'{model_path}ensemble_by_vote/{train_df_name}/dev_predicted_labels.json', 'w') as f:
        json.dump([{'id': str(k), 'labels': v} for k, v in ensemble_by_vote.items()], f, indent=4)

In [25]:
# perform the SemEval hierarchical evaluation on the predicted labels and save the output in eval folder
file_ = open(f'{model_path}ensemble_by_vote/{train_df_name}/SemEval_hierarchical_eval.txt', "w")
subprocess.Popen(f'python3 ./data/scorer-baseline/subtask_1_2a.py -g ./data/dev_gold_labels/dev_subtask1_en.json -p ./fine_tuned_models/ensemble_by_vote/{train_df_name}/dev_predicted_labels.json',shell=True,stdout=file_)
file_.close()